In [1]:
import numpy as np 
import pandas as pd

# OWN
import toolkit.cleaning_toolkit as cleaning_toolkit

In [3]:
df = pd.read_csv("../data/dataset.csv")

In [6]:
def create_date_columns(df: pd.DataFrame, date_col: str, columns_to_create: dict = None) -> pd.DataFrame:
    if not pd.api.types.is_datetime64_any_dtype(df[date_col]):
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    
    all_columns = {
        'day': lambda df: df[date_col].dt.strftime('%Y-%m-%d'),
        'week': lambda df: df[date_col].dt.to_period('W').apply(
            lambda r: r.start_time.strftime('%Y-%m-%d') if pd.notna(r) else None
        ),
        'month': lambda df: df[date_col].dt.to_period('M').apply(
            lambda r: r.start_time.strftime('%Y-%m-%d') if pd.notna(r) else None
        ),
        'year': lambda df: df[date_col].dt.to_period('Y').apply(
            lambda r: r.start_time.strftime('%Y-%m-%d') if pd.notna(r) else None
        )
    }
    
    if columns_to_create is None:
        columns_to_create = all_columns.keys()
    
    for col_name in columns_to_create:
        if col_name in all_columns:
            df[col_name] = all_columns[col_name](df)
        else:
            print(f"Advertencia: '{col_name}' no es una opción válida. Opciones válidas: {list(all_columns.keys())}")
    
    return df

In [21]:
def get_percent(
    df: pd.DataFrame, cat_col: str, group_col: str = "week"
) -> pd.DataFrame:
    total_group = df.groupby([group_col].size().reindex(week_range, fill_value=0))
    total_group_cat = df.groupby([group_col]).size().unstack(fill_value=0)
    percent_group = total_group.div(total_group_cat, axis=0).reset_index()
    
    return percent_group

In [7]:
df = create_date_columns(df, "fecha", ["week"])

In [17]:
test_date

,id_producto,id_partner,fecha,exito,franquicia,partner,valor,motivo,codigo_error
0,1,8,2024-03-02,CONFIRMED,franquicia_43,partner_43_12,11869.0,NaN,NaN
1,2,5,2024-01-27,CONFIRMED,franquicia_45,partner_45,9394.0,NaN,NaN
2,3,15,2024-07-06,CONFIRMED,NaN,partner_26,19836.0,NaN,NaN
3,4,7,2024-07-24,CONFIRMED,franquicia_11,partner_11,10666.0,NaN,NaN
4,5,17,2024-05-15,CONFIRMED,franquicia_44,partner_44,20481.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
100995,80914,8,2024-08-25,CONFIRMED,franquicia_26,partner_26,12707.0,NaN,NaN
100996,46989,6,2024-08-28,CONFIRMED,franquicia_16,NaN,NaN,NaN,NaN
100997,61697,16,2024-06-15,CONFIRMED,franquicia_26,partner_26,11846.0,NaN,NaN
100998,13172,13,2024-02-20,CONFIRMED,NaN,partner_25,27302.0,NaN,NaN
